In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Ładowanie zmiennych środowiskowych
load_dotenv()

db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_database = os.getenv("DB_DATABASE")

conn = (
    f"mssql+pyodbc://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)
engine = create_engine(conn)
print("Connection successful!")

- Temperatura powietrza
- Opady (sniegu, deszczu)
- Poziomy wod (studnie, rzeki) X
- Pokrywa śnieżna X
- Mokradla X

# Srednia temperatura od 2000 do 2024 z podziałem na miesiące

In [ ]:
# Mean temperature between 2000 and 2023
meantemp_by_year_by_month = """
WITH LiczbaMiesiecyPomiarowychNaRok AS (
    SELECT
        YEAR(d.data) AS Year,
        COUNT(DISTINCT MONTH(d.data)) AS LiczbaMiesiecyPomiarowych
    FROM
        Depesze d
    GROUP BY
        YEAR(d.data)
),
LiczbaDepeszyPerStacja AS (
    SELECT
        Stacja,
        COUNT(*) AS LiczbaDepeszy
    FROM
        Depesze d
    GROUP BY
        Stacja
)
SELECT
    YEAR(d.data) AS Year,
    MONTH(d.data) AS Month,
    AVG(TemperaturaPowietrza) AS AvgTemp
FROM
    Depesze d
WHERE
    YEAR(d.data) IN (
        SELECT Year
        FROM LiczbaMiesiecyPomiarowychNaRok
        WHERE LiczbaMiesiecyPomiarowych = 12
    ) and Stacja in (
        SELECT Stacja
        FROM LiczbaDepeszyPerStacja
        WHERE LiczbaDepeszy > 30
    )
GROUP BY
    YEAR(d.data), MONTH(d.data)
ORDER BY
    Year, Month;
"""

In [ ]:
# Pobranie danych
df = pd.read_sql_query(meantemp_by_year_by_month, engine)
df

In [ ]:
# Sprawdzenie i raportowanie zduplikowanych oraz brakujących wartości
print("Liczba zduplikowanych wierszy:", df.duplicated().sum())
print("Liczba brakujących wartości:", df.isnull().sum().sum())

# Obsługa duplikatów
df = df.drop_duplicates()

# Obsługa brakujących wartości, jeśli istnieją. W tym miejscu usuwamy wiersze z brakującymi wartościami, ale można je również wypełnić.
df = df.dropna()

# Informacja po czyszczeniu danych
print("Kształt danych po czyszczeniu:", df.shape)

# Tworzenie wykresu ```
fig = px.line(df, x='Month', y='AvgTemp', color='Year',
              title='Średnia temperatura miesięczna w ciągu lat',
              labels={
                  'Month': 'Miesiąc',
                  'AvgTemp': 'Średnia temperatura (°C)',
                  'Year': 'Rok'
              })

# Poprawa czytelności wykresu i wycentrowanie tytułu
fig.update_layout(title_x=0.5,
                  xaxis=dict(tickmode='array', tickvals=list(range(1, 13)))#,
                  # xaxis_title='Miesiąc',
                  # yaxis_title='Średnia temperatura (°C)'
                  )

# Wyświetlenie wykresu
fig.show()
plt.close()

# Porównanie globalnie średniej i globalnie mediany dla każdego miesiąca ze średnią temperatury z podziałem na rok

Średnia

In [ ]:
mean_temps_by_month = df.groupby('Month')["AvgTemp"].mean().reset_index().rename(columns={"AvgTemp": "OverallAvgTempPerMonth"})
mean_temps_by_month

In [ ]:
merged_df_with_mean = pd.merge(df, mean_temps_by_month, on='Month')
merged_df_with_mean['OverallAvgTempAvgDiff'] = merged_df_with_mean['AvgTemp'] - merged_df_with_mean['OverallAvgTempPerMonth']
merged_df_with_mean

Mediana

In [ ]:
median_temps_by_month = df.groupby('Month')["AvgTemp"].median().reset_index().rename(columns={"AvgTemp": "OverallMedianTempPerMonth"})
median_temps_by_month

In [ ]:
merged_df_with_median = pd.merge(df, median_temps_by_month, on='Month')
merged_df_with_median['OverallMedianTempAvgDiff'] = merged_df_with_median['AvgTemp'] - merged_df_with_median['OverallMedianTempPerMonth']
merged_df_with_median

In [ ]:
merged_df_with_mean['YearMonth'] = merged_df_with_mean['Year'].map(str) + "-" + merged_df_with_mean['Month'].map(str)
plt.figure(figsize=(40, 10))  # Set the figure size
plt.bar(merged_df_with_mean['YearMonth'], merged_df_with_mean['AvgTemp'], color='yellow')
plt.xticks(rotation=45)  # Rotate the x labels for better readability
plt.xticks(ticks=range(0, len(merged_df_with_mean),2),
           labels=merged_df_with_mean['YearMonth'][::2])
plt.title('Bar Chart of Average Temperature by Year-Month')
plt.xlabel('Year-Month')
plt.ylabel('Average Temperature')
plt.show()


In [ ]:
pivot_tab = merged_df_with_mean.pivot(index="Year", columns="Month",values="OverallAvgTempAvgDiff")
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_tab, annot=True, cmap="coolwarm")
plt.title("Heatmap of Temperature Difference Between Average Temperature and Global Average Temperature by Month and Year")
plt.show()

In [ ]:
newdf_med = merged_df[['YearMonth','GlobalMedianTemp']]
newdf_med

In [ ]:
newdf_med = merged_df[['GlobalMedianTemp', 'YearMonth']]

plt.figure(figsize=(40, 10))  # Set the figure size
plt.bar(newdf_med['YearMonth'], newdf_med['GlobalMedianTemp'], color='peru')
plt.xticks(rotation=45)  # Rotate the x labels for better readability
plt.xticks(ticks=range(0, len(newdf_med),2),
           labels=newdf_med['YearMonth'][::2])
plt.title('Bar Char of Median Temperature by Year-Month')
plt.xlabel('Year-Month')
plt.ylabel('Average Temperature')
plt.show()

In [ ]:
print(newdf.head())

In [ ]:
merged_df_sorted = merged_df.sort_values(by=['Year','Month'], ascending=False)
merged_df_sorted

In [ ]:
pivot_tab = merged_df_sorted.pivot(index="Year", columns="Month",values="TempDiff")
pivot_tab

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_tab, annot=True, cmap="coolwarm")
plt.title("Heatmap of Temperature Difference Between Average Temperature and Median Temperature by Month and Year")
plt.show()

In [ ]:
merged_df_positive = merged_df[merged_df['TempDiff'] > 0]
merged_df_positive.groupby('Year').size().reset_index(name='Count')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine

# Pobranie danych z bazy (zapytanie SQL do analizy opadów)
query = """
SELECT
    YEAR(data) AS Rok,
    AVG(Opady6) AS SrednieOpady6h,
    AVG(Opady12) AS SrednieOpady12h
FROM
    Synop.dbo.DaneModelTMP
WHERE
    Opady6 IS NOT NULL AND Opady12 IS NOT NULL
GROUP BY
    YEAR(data)
ORDER BY
    Rok
"""
df = pd.read_sql(query, engine)

# Wyciągnięcie danych do regresji
x = df['Rok']
y6 = df['SrednieOpady6h']
y12 = df['SrednieOpady12h']

# Obliczanie współczynników regresji dla opadów 6-godzinnych
coeff6 = np.polyfit(x, y6, 1)  # Dopasowanie liniowe
trend6 = np.polyval(coeff6, x)  # Wyliczenie wartości linii trendu

# Obliczanie współczynników regresji dla opadów 12-godzinnych
coeff12 = np.polyfit(x, y12, 1)  # Dopasowanie liniowe
trend12 = np.polyval(coeff12, x)  # Wyliczenie wartości linii trendu

# Tworzenie wykresu słupkowego
plt.figure(figsize=(12, 7))
bar_width = 0.4
plt.bar(x - bar_width / 2, y6, width=bar_width, label='Średnie Opady 6h', color='skyblue')
plt.bar(x + bar_width / 2, y12, width=bar_width, label='Średnie Opady 12h', color='orange')

# Dodanie linii trendu
plt.plot(x, trend6, color='blue', linestyle='--', linewidth=2, label='Trend Opady 6h')
plt.plot(x, trend12, color='red', linestyle='--', linewidth=2, label='Trend Opady 12h')

# Dodanie etykiet i tytułu
plt.title('Średnie Opady na przestrzeni lat z linią trendu', fontsize=16)
plt.xlabel('Rok', fontsize=14)
plt.ylabel('Średnie opady (mm)', fontsize=14)
plt.xticks(x, rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()

# Wyświetlenie wykresu
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine


# Pobranie danych z bazy (zapytanie SQL do analizy wilgotności)
query = """
SELECT
    YEAR(data) AS Rok,
    AVG(TemperaturaPunktuRosy) AS SredniaTemperaturaPunktuRosy
FROM
    Synop.dbo.DaneModelTMP
WHERE
    TemperaturaPunktuRosy IS NOT NULL and YEAR(data) BETWEEN 2000 AND 2023
GROUP BY
    YEAR(data)
ORDER BY
    Rok
"""
df = pd.read_sql(query, engine)

# Wyciągnięcie danych do regresji
x = df['Rok']
y = df['SredniaTemperaturaPunktuRosy']

# Obliczanie współczynników regresji (linia trendu)
coeff = np.polyfit(x, y, 1)  # Dopasowanie liniowe
trend = np.polyval(coeff, x)  # Wyliczenie wartości linii trendu

# Tworzenie wykresu punktowego
plt.figure(figsize=(10, 6))
plt.scatter(x, y, color='green', label='Srednia Temperatura Punktu Rosy', alpha=0.7)

# Dodanie linii trendu
plt.plot(x, trend, color='red', linestyle='--', linewidth=2, label='Linia trendu')

# Dodanie etykiet i tytułu
plt.title('Srednia temperatura punktu rosy na przestrzeni lat z linią trendu', fontsize=16)
plt.xlabel('Rok', fontsize=14)
plt.ylabel('Srednia Temperatura Punktu Rosy (°C)', fontsize=14)
plt.xticks(x, rotation=45)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()

# Wyświetlenie wykresu
plt.tight_layout()
plt.show()

In [ ]:
# Zapytanie SQL do analizy częstotliwości występowania zjawisk pogodowych
query = """
SELECT
    YEAR(data) AS Rok,
    SUM(CASE WHEN Deszcz6 = 1 THEN 1 ELSE 0 END) AS LiczbaDeszczy,
    SUM(CASE WHEN Snieg6 = 1 THEN 1 ELSE 0 END) AS LiczbaSniegów,
    SUM(CASE WHEN Burza6 = 1 THEN 1 ELSE 0 END) AS LiczbaBurz,
    SUM(CASE WHEN Mgla6 = 1 THEN 1 ELSE 0 END) AS LiczbaMgiel
FROM
    Synop.dbo.DaneModelTMP
WHERE
    Deszcz6 IS NOT NULL OR Snieg6 IS NOT NULL OR Burza6 IS NOT NULL OR Mgla6 IS NOT NULL and YEAR(data) BETWEEN 2000 AND 2023
GROUP BY
    YEAR(data)
ORDER BY
    Rok
"""
df = pd.read_sql(query, engine)
df

In [ ]:
# Wykres powierzchniowy dla zjawisk pogodowych
plt.figure(figsize=(12, 6))

plt.stackplot(df['Rok'], df['LiczbaDeszczy'], df['LiczbaSniegów'], df['LiczbaBurz'], df['LiczbaMgiel'],
              labels=['Deszcz', 'Śnieg', 'Burza', 'Mgła'], alpha=0.6)
plt.title('Trendy zjawisk pogodowych na przestrzeni lat', fontsize=16)
plt.xlabel('Rok', fontsize=14)
plt.ylabel('Liczba wystąpień', fontsize=14)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()